# Imports

In [1]:
import datetime as dt
import calendar
import time
import numpy as np
import pandas as pd
import scipy.stats as stat
import pyspark.sql.functions as functions
import math
import getpass
import pyspark
from datetime import datetime, date, timedelta
from pyspark.sql import SparkSession
import networkx as nx

spark = SparkSession \
    .builder \
    .master("yarn") \
    .appName('journey_planner-{0}'.format(getpass.getuser())) \
    .config('spark.jars.packages', 'graphframes:graphframes:0.6.0-spark2.3-s_2.11') \
    .config('spark.executor.memory', '8g') \
    .config('spark.executor.instances', '5') \
    .config('spark.port.maxRetries', '100') \
    .getOrCreate()

from graphframes import *

# Generate vertices and edges

In [20]:
# load the data
df = spark.read.csv('/datasets/sbb/2018/*/*istdaten.csv.bz2', sep=';', header=True)

In [21]:
stations = pd.read_csv('data/filtered_stations.csv')
valid_stations = set(stations['Remark'])

## Vertices

In [22]:
vertices_df = stations[['Remark', 'Longitude', 'Latitude']]
vertices_df.columns = ['id', 'lon', 'lat']
vertices = spark.createDataFrame(vertices_df)

## Walk edges

In [23]:
stations = stations[['Longitude', 'Latitude', 'Remark']];
stations['key'] = 0

earth_radius = 6371e3

def haversine(row):
    phi1         = 2 * math.pi * float(row['Latitude_x']) / 360
    phi2         = 2 * math.pi * float(row['Latitude_y']) / 360
    delta_phi    = 2 * math.pi * (float(row['Latitude_y']) - float(row['Latitude_x'])) / 360
    delta_lambda = 2 * math.pi * (float(row['Longitude_y']) - float(row['Longitude_x'])) / 360
    
    a = (math.sin(delta_phi/2) ** 2) + \
        math.cos(phi1) * math.cos(phi2) * (math.sin(delta_lambda/2) ** 2)
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    d = earth_radius * c
    
    return d / 1000

prod = pd.merge(stations, stations, on='key')
prod['dist'] = prod.apply(lambda row: haversine(row), axis=1)

In [24]:
# We don't consider walking to stops that are more than 1 kilometers away
max_walking_distance = 1
walk_df = prod[prod['dist'] <= max_walking_distance]
walk_df = walk_df[walk_df['Remark_x'] != walk_df['Remark_y']]

walk_df = walk_df[['Remark_x', 'Remark_y', 'dist']]
walk_df['type'] = 'walk'
walk_df['line'] = 'walk'
walk_df['departure_day']  = 'null'
walk_df['departure_time'] = 'null'
walk_df['arrival_time']   = 'null'
# We assume an average walking speed of 5 kilometers per hour
walk_df['lateAvg'] = walk_df.apply(lambda row: 3600 * float(row['dist']) / 5, axis=1)
walk_df['lateStd'] = 0.0
walk_df.drop('dist', axis=1, inplace=True)
walk_df.columns = ['src', 'dst', 'type', 'line', 'departure_day', 'departure_time', 'arrival_time', 'lateAvg', 'lateStd']

In [25]:
walk_edges = spark.createDataFrame(walk_df)

## Transport edges

In [26]:
dateFormat = 'dd.MM.yyyy HH:mm'
timeLate = (functions.unix_timestamp('AN_PROGNOSE', format=dateFormat)
            - functions.unix_timestamp('ANKUNFTSZEIT', format=dateFormat))

@functions.udf
def clamp(late):
    return 0 if late < 0 else late

valid_stops = df.filter((df.DURCHFAHRT_TF=='false') & 
                        (df.FAELLT_AUS_TF=='false') & 
                        (df.ZUSATZFAHRT_TF=='false') &
                        (df.AN_PROGNOSE_STATUS=='GESCHAETZT') &
                        (df.HALTESTELLEN_NAME.isin(valid_stations))) \
                .select('BETRIEBSTAG',
                        'FAHRT_BEZEICHNER', 
                        'PRODUKT_ID', 
                        'LINIEN_TEXT', 
                        'HALTESTELLEN_NAME', 
                        'AN_PROGNOSE',
                        'ANKUNFTSZEIT', 
                        'ABFAHRTSZEIT') \
                .withColumn('AN_PROGNOSE',  functions.to_timestamp(df.AN_PROGNOSE, dateFormat))  \
                .withColumn('ANKUNFTSZEIT', functions.to_timestamp(df.ANKUNFTSZEIT, dateFormat)) \
                .withColumn('ABFAHRTSZEIT', functions.to_timestamp(df.ABFAHRTSZEIT, dateFormat)) \
                .withColumn('late', clamp(timeLate)) \
                .drop('AN_PROGNOSE')

In [27]:
departures = valid_stops.filter(valid_stops.ABFAHRTSZEIT.isNotNull())\
                        .drop('ANKUNFTSZEIT', 'late')
arrivals   = valid_stops.filter(valid_stops.ANKUNFTSZEIT.isNotNull())\
                        .drop('ABFAHRTSZEIT')

In [28]:
arrivals.createOrReplaceTempView('arrivals')
departures.createOrReplaceTempView('departures')

joinQuery = 'SELECT d.HALTESTELLEN_NAME AS src, a.HALTESTELLEN_NAME AS dst,              \
                    d.PRODUKT_ID AS type, d.LINIEN_TEXT AS line,                         \
                    date_format(d.ABFAHRTSZEIT, \'EEEE\') AS departure_day,              \
                    SUBSTRING(d.ABFAHRTSZEIT, 12, 8) AS departure_time,                  \
                    SUBSTRING(a.ANKUNFTSZEIT, 12, 8) AS arrival_time,                    \
                    a.late                                                               \
             FROM arrivals AS a INNER JOIN departures AS d                               \
             ON a.BETRIEBSTAG == d.BETRIEBSTAG                                           \
             AND a.FAHRT_BEZEICHNER == d.FAHRT_BEZEICHNER                                \
             WHERE a.HALTESTELLEN_NAME != d.HALTESTELLEN_NAME                            \
             AND d.ABFAHRTSZEIT < a.ANKUNFTSZEIT'

edges = spark.sql(joinQuery)

In [29]:
edges.createOrReplaceTempView('edges')

query = 'SELECT src, dst, type, line, departure_day, departure_time, arrival_time,              \
         AVG(late) AS lateAvg, STD(late) AS lateStd                                             \
         FROM edges GROUP BY src, dst, type, line, departure_day, departure_time, arrival_time'

aggregated = spark.sql(query)
aggregated_edges = aggregated.na.fill(0.0)

all_edges = aggregated_edges.union(walk_edges)

## Write data to hdfs

In [30]:
all_edges.write.parquet('/homes/schmutz/edges', mode='overwrite')

Py4JJavaError: An error occurred while calling o1457.parquet.
: org.apache.hadoop.security.AccessControlException: Permission denied: user=kgerard, access=WRITE, inode="/homes/schmutz":schmutz:hadoop:drwxr-xr-x
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:399)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:258)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:193)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1857)
	at org.apache.hadoop.hdfs.server.namenode.FSDirDeleteOp.delete(FSDirDeleteOp.java:110)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.delete(FSNamesystem.java:3002)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.delete(NameNodeRpcServer.java:1095)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.delete(ClientNamenodePr

In [ ]:
vertices.write.parquet('/homes/schmutz/vertices', mode='overwrite')

## Load data from hdfs

In [2]:
vertices = spark.read.parquet('/homes/schmutz/vertices')

In [4]:
vertices.show(n=5)

+--------------------+-----------------+-----------------+
|                  id|              lon|              lat|
+--------------------+-----------------+-----------------+
|   Zumikon, Gössikon|         8.614773|        47.332474|
|   Zumikon, Waltikon|         8.618188|        47.336109|
|Zumikon, Dorfzentrum|         8.622922|        47.332976|
|Zürich, Meierhofp...|         8.499375|        47.402009|
|  Zürich, Heizenholz|8.483903999999999|47.41229600000001|
+--------------------+-----------------+-----------------+
only showing top 5 rows



In [3]:
edges = spark.read.parquet('/homes/schmutz/edges')

In [118]:
edges.show(n=5)

+--------------------+--------------------+----+----+-------------+--------------+------------+-----------------+-------+
|                 src|                 dst|type|line|departure_day|departure_time|arrival_time|          lateAvg|lateStd|
+--------------------+--------------------+----+----+-------------+--------------+------------+-----------------+-------+
|Zürich, Carl-Spit...|Zürich, Stodolast...|walk|walk|         null|          null|        null|243.1882193019385|    0.0|
|Zürich, Carl-Spit...|      Zürich, Rehalp|walk|walk|         null|          null|        null|  616.09711715276|    0.0|
|Zürich, Carl-Spit...|     Zürich, Segeten|walk|walk|         null|          null|        null|401.5413867498982|    0.0|
|Zürich, Carl-Spit...|Zürich, Waserstrasse|walk|walk|         null|          null|        null|227.3430054944393|    0.0|
|Zürich, Carl-Spit...| Zürich, Wiesliacher|walk|walk|         null|          null|        null|607.4434392971077|    0.0|
+--------------------+--

In [4]:
graph = GraphFrame(vertices, edges)

# Naive Journey Planner

In [49]:
MINUTES_PER_DAY = 1440
MINUTES_PER_HOUR = 60
SECONDS_PER_MINUTE = 60

def computeDiff(departure, arrival):
    dep = (departure[2:]).split(':')
    arr = (arrival[2:]).split(':')
    a = (int(arrival[:1]) - int(departure[:1])) * MINUTES_PER_DAY
    b = (int(arr[0]) - int(dep[0])) * MINUTES_PER_HOUR
    c = (int(arr[1]) - int(dep[1]))
    tot = a + b + c
    hours = tot // MINUTES_PER_HOUR
    minutes = tot % MINUTES_PER_HOUR
    return "{:02d}".format(int(hours)) + ':' + "{:02d}".format(int(minutes)) + ':00'

def computeCost(cost, late):
    tmp = cost.split(':')
    a = int(tmp[0][2:]) * MINUTES_PER_HOUR + int(tmp[1])
    b = late // SECONDS_PER_MINUTE
    prefix = tmp[0][:2] if a > b else '0-'
    a = (a - b) % MINUTES_PER_DAY
    minutes = a % MINUTES_PER_HOUR
    hours = (a - minutes) // MINUTES_PER_HOUR
    return prefix + "{:02d}".format(int(hours)) + ':' + "{:02d}".format(int(minutes)) + ':00'

def getFilteredEdges(startDay, finishDay, startTime, finishTime, duration):
    def valid(day, depTime, arrTime, walkTime):
        if startDay==finishDay:
            return ((day=='null') & (walkTime<=duration)) | \
                    ((day==startDay) & (depTime>=startTime) & (arrTime<=finishTime) & (depTime<=arrTime))
        else:
            return ((day=='null') & (walkTime<=duration)) | \
                    (((day==startDay) & (depTime>=startTime) & ((depTime<=arrTime) | (arrTime<=finishTime))) | \
                     ((day==finishDay) & (depTime<finishTime) & (arrTime<=finishTime)))

    return graph.filterEdges(valid(graph.edges.departure_day, 
                                graph.edges.departure_time,
                                graph.edges.arrival_time,
                                graph.edges.lateAvg)).edges

def add_vertice_to_set(max_set, vertice, vertice_costs, edges, next_vertices):
    
    max_set.add(vertice)
    cost = vertice_costs[vertice]
    
    vertice_edges = edges.in_edges(vertice, data=True)
    
    for parallel_paths in vertice_edges:
        i = 0
        edge = parallel_paths[2]['edge']
        if edge['type'] == 'walk':
            new_cost = computeCost(cost, edge['lateAvg'])
            if edge['src'] not in vertice_costs or new_cost > vertice_costs[edge.dst]:
                next_vertices[edge['src']] = edge
                vertice_costs[edge.src] = new_cost
        elif edge['arrival_time'] < cost and (edge['src'] not in vertice_costs or edge['departure_time'] > vertice_costs[edge['dst']]):
            vertice_costs[edge['src']] = edge['departure_time']
            next_vertices[edge['src']] = edge
        i += 1

def get_max_vertice_not_in_set(max_set, vertice_costs, min_trip_departure_time):
    max_vertice = None
    max_cost = min_trip_departure_time
    for vertice in vertice_costs:
        if vertice not in max_set and vertice_costs[vertice] > max_cost:
            max_cost = vertice_costs[vertice]
            max_vertice = vertice
    
    return max_vertice

def find_path(next_vertices, current_vertice, current_path):
    if current_vertice not in next_vertices:
        return current_path
    next_vertice = next_vertices[current_vertice]['dst']
    current_path.append(next_vertices[current_vertice])
    return find_path(next_vertices, next_vertice, current_path)
    

def find_shortest_path(departure_station, arrival_station, 
                       startDateTime, endDateTime, 
                       min_probability_of_sucess, get_all_destinations=False,
                       subgraph=None):
    
    print(startDateTime)
    print(endDateTime)
    
    startTime  = str(startDateTime.time())
    endTime = str(endDateTime.time())

    startDay  = calendar.day_name[startDateTime.weekday()]
    endDay = calendar.day_name[endDateTime.weekday()]
    
    min_trip_departure_time = '0-' + startTime
    
    endTimePrefix = '0-' if (startDay == endDay) else '1-'
    requested_arrival_time = endTimePrefix + endTime
    
    duration = (endDateTime - startDateTime).seconds
    
    print(startDay, startTime)
    print(endDay, endTime)
    
    if subgraph is None:
        filtered_edges = getFilteredEdges(startDay, endDay, startTime, endTime, duration).toPandas()

        def to_dt(time):
            if time == 'null':
                return 'null'
            elif time >= startTime:
                return '0-' + time
            else:
                return '1-' + time



        filtered_edges['departure_time'] = filtered_edges['departure_time'].map(lambda x: to_dt(x))
        filtered_edges['arrival_time']   = filtered_edges['arrival_time'].map(lambda x: to_dt(x))

        G = nx.MultiDiGraph()

        for i, edge in filtered_edges.iterrows():
            G.add_edge(edge['src'], edge['dst'], edge=edge)
    else:
        G = subgraph
    
    # as day#-hh-mm-ss
    vertice_costs = {}
    vertice_costs[arrival_station] = requested_arrival_time

    max_set = set()
    next_vertices = {}
    add_vertice_to_set(max_set, arrival_station, vertice_costs, G, next_vertices)
    no_solution = False
    print("data loaded", time.time() - starttime)
    while((departure_station not in max_set or get_all_destinations) and not no_solution):
        max_vertice = get_max_vertice_not_in_set(max_set, vertice_costs, min_trip_departure_time)
        if max_vertice is None:
            no_solution = True
        else:
            add_vertice_to_set(max_set, max_vertice, vertice_costs, G, next_vertices)
    
    if get_all_destinations:
        return vertice_costs
    if no_solution:
        return "no solution"
    departure_time = (vertice_costs[departure_station])[2:]
        
    trip_duration = computeDiff(vertice_costs[departure_station], requested_arrival_time)
    
    return departure_time, trip_duration, find_path(next_vertices, departure_station, [departure_station])

In [141]:
fromStation = 'Kilchberg'
toStation   = 'Urdorf, Schlierenstrasse'
startDateTime = datetime(2019, 6, 3, 18, 16)
endDateTime   = datetime(2019, 6, 3, 20, 57)
starttime = time.time()
res = find_shortest_path(fromStation, toStation, 
                   startDateTime, 
                   endDateTime, 0, True)
print(time.time() - starttime)
res

2019-06-03 18:16:00
2019-06-03 20:57:00
Monday 18:16:00
Monday 20:57:00
data loaded 7.144065618515015
8.136763095855713


{'Urdorf, Schlierenstrasse': '0-20:57:00',
 'Dietikon, Birmensdorferstrasse': '0-20:47:00',
 'Dietikon, Schönenwerd': '0-20:49:00',
 'Glanzenberg, Bahnhof': '0-20:47:00',
 'Urdorf': '0-20:47:00',
 'Glanzenberg': '0-20:47:00',
 'Schlieren, Spital Limmattal': '0-20:51:00',
 'Schlieren, Steinacker': '0-20:50:00',
 'Urdorf, Bahnhof': '0-20:46:00',
 'Urdorf, Feldegg': '0-20:54:00',
 'Urdorf, Herweg': '0-20:52:00',
 'Urdorf, Luberzen': '0-20:54:00',
 'Urdorf, Neumatt': '0-20:52:00',
 'Urdorf, Neumattstrasse': '0-20:52:00',
 'Urdorf, Niederurdorf': '0-20:53:00',
 'Urdorf, Rainweg': '0-20:54:00',
 'Urdorf, Spitzacker': '0-20:48:00',
 'Schlieren, Heimeliweg': '0-20:45:00',
 'Schlieren, Reitmen': '0-20:46:00',
 'Urdorf, Sonne': '0-20:43:00',
 'Urdorf, Uitikonerstrasse': '0-20:45:00',
 'Urdorf, Oberurdorf': '0-20:41:00',
 'Urdorf Weihermatt': '0-20:42:00',
 'Urdorf Weihermatt, Bahnhof': '0-20:41:00',
 'Schlieren, Kesslerstrasse': '0-20:42:00',
 'Schlieren, Nassacker': '0-20:40:00',
 'Wallisellen'

In [43]:
a = None

In [45]:
if a == None:
    print('bla')

bla


In [52]:
MINUTES_PER_DAY = 1440
MINUTES_PER_HOUR = 60
SECONDS_PER_MINUTE = 60

MAX_PATH_LENGTH = 4

def computeLengthInMinutes(departure, arrival):
    dep = (departure[2:]).split(':')
    arr = (arrival[2:]).split(':')
    a = (int(arrival[:1]) - int(departure[:1])) * MINUTES_PER_DAY
    b = (int(arr[0]) - int(dep[0])) * MINUTES_PER_HOUR
    c = (int(arr[1]) - int(dep[1]))
    return a + b + c

def computeCost(cost, late):
    tmp = cost.split(':')
    a = int(tmp[0][2:]) * MINUTES_PER_HOUR + int(tmp[1])
    b = late // SECONDS_PER_MINUTE
    prefix = tmp[0][:2] if a > b else '0-'
    a = (a - b) % MINUTES_PER_DAY
    minutes = a % MINUTES_PER_HOUR
    hours = (a - minutes) // MINUTES_PER_HOUR
    return prefix + "{:02d}".format(int(hours)) + ':' + "{:02d}".format(int(minutes)) + ':00'

def getFilteredEdges(startDay, finishDay, startTime, finishTime, duration):
    def valid(day, depTime, arrTime, walkTime):
        if startDay==finishDay:
            return ((day=='null') & (walkTime<=duration)) | \
                    ((day==startDay) & (depTime>=startTime) & (arrTime<=finishTime) & (depTime<=arrTime))
        else:
            return ((day=='null') & (walkTime<=duration)) | \
                    (((day==startDay) & (depTime>=startTime) & ((depTime<=arrTime) | (arrTime<=finishTime))) | \
                     ((day==finishDay) & (depTime<finishTime) & (arrTime<=finishTime)))

    return graph.filterEdges(valid(graph.edges.departure_day, 
                                graph.edges.departure_time,
                                graph.edges.arrival_time,
                                graph.edges.lateAvg)).edges

def computeTime(start, duration):
    tmp = start.split(':')
    a = int(tmp[0][2:]) * MINUTES_PER_HOUR + int(tmp[1])
    b = duration // SECONDS_PER_MINUTE
    prefix = tmp[0][:2] if a + b < MINUTES_PER_DAY else '1-'
    a = (a + b) % MINUTES_PER_DAY
    minutes = a % MINUTES_PER_HOUR
    hours = (a - minutes) // MINUTES_PER_HOUR
    return prefix + "{:02d}".format(int(hours)) + ':' + "{:02d}".format(int(minutes)) + ':00'


def compute_paths_between_naive(src, dst, edges, visited, curr_path, curr_time, 
                                requested_arrival_time, paths, last_line_taken, max_dep_times):
    visited.add(src)
    
    if src == dst:
        print('path found')
        final_path = curr_path.copy()
        final_path.append(curr_time)
        paths.append(final_path)
    elif len(curr_path) < MAX_PATH_LENGTH:
        vertice_edges = edges.out_edges(src, data=True)
        for vertice_edge in vertice_edges:
            edge = vertice_edge[2]['edge']
            if edge['dst'] not in visited and edge['line'] != last_line_taken:
                if edge['type'] == 'walk':
                    new_time = computeTime(curr_time, edge['lateAvg'])
                    if new_time <= requested_arrival_time and new_time < max_dep_times[edge['dst']]:
                        curr_path.append(edge)
                        compute_paths_between_naive(edge['dst'], dst, edges, visited, curr_path, 
                                                    new_time, requested_arrival_time, paths, 
                                                    edge['line'], max_dep_times)
                        curr_path.pop();
                elif edge['departure_time'] >= curr_time and edge['arrival_time'] < max_dep_times[edge['dst']]:
                    curr_path.append(edge)
                    compute_paths_between_naive(edge['dst'], dst, edges, visited, curr_path, 
                                                edge['arrival_time'], requested_arrival_time, 
                                                paths, edge['line'], max_dep_times)
                    curr_path.pop();
        
    visited.remove(src)

def computeProb(depTime, lateAvg, lateStd, arrTime):
    length = computeLengthInMinutes(depTime, arrTime) * 60
    if lateStd != 0.0:
        return stat.norm(loc=lateAvg, scale=lateStd).cdf(length)
    elif lateAvg <= length:
        return 1.0
    else:
        return 0.0
    

def compute_paths_between(src, dst, edges, visited, curr_path, curr_prob, curr_time, curr_lateAvg, curr_lateStd, 
                          requested_arrival_time, paths, last_line_taken, max_dep_times, min_prob_success):
    visited.add(src)
    
    if src == dst:
        final_prob = computeProb(curr_time, curr_lateAvg, curr_lateStd, requested_arrival_time) * curr_prob
        if final_prob >= min_prob_success:
            final_path = curr_path.copy()
            final_path.append(curr_time)
            final_path.append(final_prob)
            paths.append(final_path)
            
    elif len(curr_path) < MAX_PATH_LENGTH:
        vertice_edges = edges.out_edges(src, data=True)
        for vertice_edge in vertice_edges:
            edge = vertice_edge[2]['edge']
            if edge['dst'] not in visited and edge['line'] != last_line_taken:
                if edge['type'] == 'walk':
                    new_time = computeTime(curr_time, edge['lateAvg'])
                    if new_time <= requested_arrival_time and new_time < max_dep_times[edge['dst']]:
                        curr_path.append(edge)
                        compute_paths_between(edge['dst'], dst, edges, visited, curr_path, 
                                              curr_prob, new_time, curr_lateAvg, curr_lateStd, 
                                              requested_arrival_time, paths, 
                                              edge['line'], max_dep_times, min_prob_success)
                        curr_path.pop();
                elif edge['departure_time'] >= curr_time and edge['arrival_time'] < max_dep_times[edge['dst']]:
                    prob = computeProb(curr_time, curr_lateAvg, curr_lateStd, edge['departure_time'])
                    new_prob = curr_prob * prob
                    if new_prob >= min_prob_success:
                        curr_path.append(edge)
                        compute_paths_between(edge['dst'], dst, edges, visited, curr_path, 
                                              new_prob, edge['arrival_time'], edge['lateAvg'], edge['lateStd'],
                                              requested_arrival_time, paths, 
                                              edge['line'], max_dep_times, min_prob_success)
                        curr_path.pop();
        
    visited.remove(src)
    
    
def dfs(departure_station, arrival_station, 
        startDateTime, endDateTime, 
        min_probability_of_sucess):
    
    print(startDateTime)
    print(endDateTime)
    
    startTime  = str(startDateTime.time())
    endTime = str(endDateTime.time())

    startDay  = calendar.day_name[startDateTime.weekday()]
    endDay = calendar.day_name[endDateTime.weekday()]
    
    min_trip_departure_time = '0-' + startTime
    
    endTimePrefix = '0-' if (startDay == endDay) else '1-'
    requested_arrival_time = endTimePrefix + endTime
    
    duration = (endDateTime - startDateTime).seconds
    
    print(startDay, startTime)
    print(endDay, endTime)
    
    filtered_edges = getFilteredEdges(startDay, endDay, startTime, endTime, duration).toPandas()
    
    def to_dt(time):
        if time == 'null':
            return 'null'
        elif time >= startTime:
            return '0-' + time
        else:
            return '1-' + time


    
    filtered_edges['departure_time'] = filtered_edges['departure_time'].map(lambda x: to_dt(x))
    filtered_edges['arrival_time']   = filtered_edges['arrival_time'].map(lambda x: to_dt(x))
    
    G = nx.MultiDiGraph()
    
    for i, edge in filtered_edges.iterrows():
        G.add_edge(edge['src'], edge['dst'], edge=edge)
    
    print("data loaded", time.time() - starttime)
    
    max_dep_times = find_shortest_path(departure_station, arrival_station, 
                                       startDateTime, 
                                       endDateTime, min_probability_of_sucess, True, 
                                       subgraph=G)
    
    visited = set()
    best_times = {}
    curr_time = min_trip_departure_time
    curr_path = [departure_station]
    paths = []
    
    #compute_paths_between_naive(departure_station, arrival_station, G, 
    #                            visited, curr_path, curr_time, 
    #                            requested_arrival_time, paths, '', max_dep_times)
    
    compute_paths_between(departure_station, arrival_station, G, 
                          visited, curr_path, 1.0, curr_time, 0.0, 0.0, 
                          requested_arrival_time, paths, 'walk', max_dep_times, 
                          min_probability_of_sucess)
    
    times = [computeDiff(path[1]['departure_time'], path[-2]) for path in paths]
    best_path_idx = np.argmin(times)
    best_path = paths[best_path_idx]
    
    return {'departure time' : best_path[1]['departure_time'][2:], 
           'arrival_time' : best_path[-2][2:], 'duration' : times[best_path_idx]}, best_path[:-2]

In [53]:
fromStation = 'Kilchberg'
toStation   = 'Urdorf, Schlierenstrasse'
startDateTime = datetime(2019, 6, 3, 18, 16)
endDateTime   = datetime(2019, 6, 3, 20, 57)
starttime = time.time()
res = dfs(fromStation, toStation, 
          startDateTime, 
          endDateTime, 0.95)
print('total runtime:', time.time() - starttime)
res

2019-06-03 18:16:00
2019-06-03 20:57:00
Monday 18:16:00
Monday 20:57:00
data loaded 3.0884201526641846
2019-06-03 18:16:00
2019-06-03 20:57:00
Monday 18:16:00
Monday 20:57:00
data loaded 3.089740037918091
total runtime: 11.44856595993042


({'arrival_time': '19:05:00',
  'departure time': '18:26:00',
  'duration': '00:39:00'},
 ['Kilchberg', src                Kilchberg
  dst                Zürich HB
  type                     Zug
  line                     S24
  departure_day         Monday
  departure_time    0-18:26:00
  arrival_time      0-18:39:00
  lateAvg              63.3333
  lateStd              75.5373
  Name: 21136, dtype: object, src                 Zürich HB
  dst               Glanzenberg
  type                      Zug
  line                      S12
  departure_day          Monday
  departure_time     0-18:44:00
  arrival_time       0-18:55:00
  lateAvg                   130
  lateStd               77.6872
  Name: 21706, dtype: object, src                            Glanzenberg
  dst               Urdorf, Schlierenstrasse
  type                                  walk
  line                                  walk
  departure_day                         null
  departure_time                        null
  arr

# ARCHIVED CODE FOR POSSIBLE REUSE

In [62]:
def getSubGraph(graph, startDay, finishDay, startTime, finishTime, duration):
    def valid(day, depTime, arrTime, walkTime):
        if startDay==finishDay:
            return ((day=='null') & (walkTime<=duration)) | \
                    ((day==startDay) & (depTime>=startTime) & (arrTime<=finishTime) & (depTime<=arrTime))
        else:
            return ((day=='null') & (walkTime<=duration)) | \
                    (((day==startDay) & (depTime>=startTime) & ((depTime<=arrTime) | (arrTime<=finishTime))) | \
                     ((day==finishDay) & (depTime<finishTime) & (arrTime<=finishTime)))

    return graph.filterEdges(valid(graph.edges.departure_day, 
                                graph.edges.departure_time,
                                graph.edges.arrival_time,
                                graph.edges.lateAvg))  \
                .dropIsolatedVertices()

def howFarNaive(graph, fromStation, startDateTime, duration):
    
    if duration >= 120:
        print('You can walk anywhere in that time')
        return
    
    finishDateTime = startDateTime + timedelta(minutes=duration)

    startTime  = str(startDateTime.time())
    finishTime = str(finishDateTime.time())

    startDay  = calendar.day_name[startDateTime.weekday()]
    finishDay = calendar.day_name[finishDateTime.weekday()]

    print(startDay, startTime)
    print(finishDay, finishTime)
    
    @functions.udf
    def addTime(arr_time, dep_time, late):
        if arr_time=='null':
            tmp = dep_time.split(':')
            return str((datetime.combine(date.today(), dt.time(int(tmp[0]), int(tmp[1]), int(tmp[2]))) + 
                        timedelta(seconds=int(late))).time())
        else:
            return arr_time
    
    @functions.udf
    def checkDay(day, dep_time, arr_time):
        return finishDay if arr_time<dep_time else day
    
    @functions.udf
    def checkWalk(ttype):
        return 1 if ttype=='walk' else 0
    
    @functions.udf
    def checkIfValid(arr_time, day):
        tmp = arr_time.split(':')
        arr_date = startDateTime.date() if day==calendar.day_name[startDateTime.weekday()] else finishDateTime.date()
        arrival = datetime.combine(arr_date, dt.time(int(tmp[0]), int(tmp[1]), int(tmp[2])))
        return arrival < finishDateTime
    
    reachable = vertices.filter(vertices.id==fromStation)             \
                        .withColumn('time', functions.lit(startTime)) \
                        .withColumn('day', functions.lit(startDay))   \
                        .withColumn('just_walked', functions.lit(0))
    
    g = getSubGraph(graph, startDay, finishDay, startTime, finishTime, 60*duration)
    g.persist();
    g.edges.createOrReplaceTempView('edges')
    g.vertices.createOrReplaceTempView('vertices')
    
    curr = reachable
    
    #while len(curr.head(1)) > 0:
    for i in range(1):
        curr.createOrReplaceTempView('curr')

        query = 'SELECT v.*, c.time AS past_time, c.just_walked, c.day, e.type,          \
                        e.departure_time, e.arrival_time, e.lateAvg                      \
                 FROM curr AS c INNER JOIN edges AS e INNER JOIN vertices AS v           \
                 ON c.id==e.src                                                          \
                 AND e.dst==v.id                                                         \
                 WHERE (e.type!=\'walk\' OR c.just_walked==0)                            \
                 AND (e.type==\'walk\'                                                   \
                 OR (e.departure_time>=c.time AND c.day==e.departure_day)                \
                 OR (e.departure_time<c.time AND c.day!=e.departure_day))'

        curr = spark.sql(query).withColumn('time', addTime('arrival_time', 'past_time', 'lateAvg')) \
                               .withColumn('day', checkDay('day', 'past_time', 'time'))             \
                               .filter(checkIfValid('time', 'day')=='true')                         \
                               .withColumn('just_walked', checkWalk('type'))                        \
                               .select('id', 'lon', 'lat', 'time', 'day', 'just_walked')
        curr.persist()
        reachable = reachable.union(curr)
    
    @functions.udf
    def computeRadius(arr_time, day):
        tmp = arr_time.split(':')
        arr_date = startDateTime.date() if day==calendar.day_name[startDateTime.weekday()] else finishDateTime.date()
        arrival = datetime.combine(arr_date, dt.time(int(tmp[0]), int(tmp[1]), int(tmp[2])))
        return (finishDateTime - arrival).seconds * 5 // 3.6
    
    reachable = reachable.withColumn('radius', computeRadius('time', 'day')) \
                         .select('id', 'lon', 'lat', 'radius', 'time', 'just_walked')                     \
                         .toPandas()
    
    g.unpersist();
    
    return reachable

In [65]:
graph = GraphFrame(vertices, edges)
fromStation = 'Dietlikon'
startDateTime  = datetime(2019, 5, 31, 23, 45)
duration = 11

start = time.time()
reachable = howFarNaive(graph, fromStation, startDateTime, duration)
print(time.time() - start)
reachable

Friday 23:45:00
Friday 23:56:00
9.501955032348633


,id,lon,lat,radius,time,just_walked
0,Dietlikon,8.619255,47.420195,916.0,23:45:00,0
1,"Dietlikon, Bahnhof",8.619087,47.420359,895.0,23:45:15,1
2,"Brüttisellen, Gsellhof",8.629667,47.421708,116.0,23:54:36,1
3,"Dietlikon, Bahnhof/Bad",8.620805,47.421960,688.0,23:47:44,1
4,"Dietlikon, Brandbachstrasse",8.624657,47.416965,374.0,23:51:30,1
5,"Dietlikon, Dornenstrasse",8.616896,47.416585,477.0,23:50:16,1
6,"Dietlikon, Dübendorferstrasse",8.619558,47.413509,173.0,23:53:55,1
7,"Dietlikon, Hofwiesen",8.618518,47.426391,226.0,23:53:17,1
8,"Dietlikon, Industriestrasse",8.621946,47.414249,226.0,23:53:17,1
9,"Dietlikon, Fuchshalde",8.612612,47.420029,416.0,23:51:00,1
